<a href="https://colab.research.google.com/github/amey-joshi/am/blob/master/optim/Multi_objective_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

from sklearn.preprocessing import StandardScaler
from scipy import optimize

# Generate simulated data

We simulate data assuming that all the variables are normally distributed. The mean and the standard deviations of the variables are assumed to take the values in the variables <code>xmeans</code> and <code>xstds</code>.

In [2]:
n_obs = 1160 # Number of observations
xmeans = [0.15, 130.84, 85.73, 1046.68, 320.59]
xstds  = [0.014, 14.29, 8.79, 114.32, 25.43]

np.random.seed(11112020)
x1 = np.random.normal(xmeans[0], xstds[0], n_obs)
x2 = np.random.normal(xmeans[1], xstds[1], n_obs)
x3 = np.random.normal(xmeans[2], xstds[2], n_obs)
x4 = np.random.normal(xmeans[3], xstds[3], n_obs)
x5 = np.random.normal(xmeans[4], xstds[4], n_obs)

ymeans = [282.38, 107.12, 0.01, 0.38, 1.31]
ystds  = [27.21, 23.57, 0.006, 0.07, 0.09]

y1 = np.random.normal(ymeans[0], ystds[0], n_obs)
y2 = np.random.normal(ymeans[1], ystds[1], n_obs)
y3 = np.random.normal(ymeans[2], ystds[2], n_obs)
y4 = np.random.normal(ymeans[3], ystds[3], n_obs)
y5 = np.random.normal(ymeans[4], ystds[4], n_obs)

data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5,
                     'y1': y1, 'y2': y2, 'y3': y3, 'y4': y4, 'y5': y5,})

In the sections to follow, we will predict $y_i$ as functions of $x_1, \ldots, x_5$ assuming a variety of functional forms of the dependency. We will start with the linear function and then try a few non-linear ones.

# Common functions

In [3]:
def compute_error_orig(X, Y, beta, scaler, X1 = None):
  """ Compute the error between unscaled variables. 
  
  Some times, we run a model on scaled variables. In that case, we apply the
  inverse transform on the results and compare them with the original data.
  """
  if X1 is None:
    Y_hat = cp.matmul(X, beta)
  else:
    Y_hat = cp.matmul(X1, beta)

  Z = np.concatenate((X, Y_hat.value), axis=1)
  results = pd.DataFrame(data = scaler.inverse_transform(Z))
  results.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'y1', 'y2', 'y3', 'y4', 'y5']
  error = (data - results)**2
  total_errors = error.sum().to_numpy()[5:]
  rms_error = np.sqrt(total_errors)/n_obs
  print(rms_error)

  return rms_error

In [4]:
def compute_error(X, Y, beta):
  """
  Compute error between predicted and actual response. 
  """
  Y_hat = cp.matmul(X, beta)
  error = Y - Y_hat
  error_sq = np.square(error.value)
  sse = np.sum(error_sq, axis=0)
  rms = np.sqrt(sse)/n_obs
  print(f'Total sum of squares of error: {sse}')
  print(f'Root mean square of error: {rms}')

  return rms

In [5]:
def print_status(prob, beta):
  """ Prints the status of the optimizer. """
  print(f'Problem status: {prob.status}')
  print(f'Optimal value: {prob.value}')
  print('Beta values:')
  print(beta.value)

This data frame will store the RMS errors of each model.

In [6]:
column_names = ['model_name', 'err_y1', 'err_y2', 'err_y3', 'err_y4', 'err_y5']
model_summary = pd.DataFrame(columns=column_names)

# Linear model - 1

We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5
\end{eqnarray}

Optimizers work best when the variables are scaled. We use the scikit-learn scaler to transform our data. 

In [7]:
scaler = StandardScaler()
scaler.fit(data)
scaled_data = scaler.transform(data)

We segregate the preditors into the matrix <code>X</code> and the responses into the matrix <code>Y</code>.

In [8]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

We now build a linear model and run it. The variable <code>beta</code> has the coeffifients of the linear model.

**This step can take a few minutes.**

In [ ]:
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

35.5324071011577

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 35.5324071011577
Beta values:
[[ 0.0109488   0.02589171 -0.0442837   0.02470366 -0.02229602]
 [ 0.00994741 -0.03560378  0.02444012  0.01066956  0.0111661 ]
 [-0.0201376   0.02608267  0.02757303 -0.00720288  0.03756629]
 [ 0.03318335 -0.01143796 -0.01100121  0.02615937 -0.05206846]
 [ 0.03577049 -0.04468985  0.01498306 -0.02103517 -0.02175626]]


In [ ]:
Y_hat = cp,matmul(X, beta) # Estimates of Y

In [ ]:
compute_error(X, Y, beta)

Total sum of squares of error: [1156.61287274 1154.53861065 1155.74789072 1157.75610247 1154.04109214]
Root mean square of error: [0.02931811 0.02929181 0.02930715 0.0293326  0.0292855 ]


array([0.02931811, 0.02929181, 0.02930715, 0.0293326 , 0.0292855 ])

In [ ]:
errors = compute_error_orig(X, Y, beta, scaler)

[7.88313341e-01 6.92319500e-01 1.75097276e-04 2.01302848e-03
 2.63304027e-03]


In [ ]:
model_summary = model_summary.append({'model_name': 'Linear model - 1', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Linear model - 2
We will fit a linear model with an intercept.

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 + \beta_{10}\\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 + \beta_{20} \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 + \beta_{30} \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 + \beta_{40} \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5 + \beta_{50}
\end{eqnarray}

In [ ]:
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X1, beta)))
prob = cp.Problem(obj)
prob.solve()

35.535277027584655

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 35.535277027584655
Beta values:
[[ 1.09397889e-02  2.59220080e-02 -4.43067351e-02  2.48319614e-02
  -2.21265280e-02]
 [ 1.00519081e-02 -3.56046136e-02  2.47613889e-02  1.09339153e-02
   1.12205095e-02]
 [-2.02438936e-02  2.62953456e-02  2.76433063e-02 -7.35515989e-03
   3.72673410e-02]
 [ 3.30284669e-02 -1.12285498e-02 -1.08815323e-02  2.60963434e-02
  -5.23252671e-02]
 [ 3.59263790e-02 -4.47787525e-02  1.50764993e-02 -2.11889100e-02
  -2.19615602e-02]
 [ 9.43465094e-05  6.67892246e-06  1.90589107e-04  1.46015662e-04
   3.20768804e-05]]


In [ ]:
Y_hat = cp,matmul(X1, beta) # Estimates of Y

In [ ]:
compute_error(X1, Y, beta)

Total sum of squares of error: [1156.61297354 1154.53873561 1155.74810773 1157.75627625 1154.04137772]
Root mean square of error: [0.02931811 0.02929181 0.02930715 0.0293326  0.0292855 ]


array([0.02931811, 0.02929181, 0.02930715, 0.0293326 , 0.0292855 ])

In [ ]:
errors = compute_error_orig(X, Y, beta, scaler, X1)

[7.88313375e-01 6.92319537e-01 1.75097293e-04 2.01302863e-03
 2.63304059e-03]


In [ ]:
model_summary = model_summary.append({'model_name': 'Linear model - 2', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 1

We fit the model:

\begin{eqnarray}
\ln y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 \\
\ln y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 \\
\end{eqnarray}

In [ ]:
data_nm1 = data.copy()

data_nm1['x2'] = np.log(data_nm1['x2'])
data_nm1['x3'] = np.log(data_nm1['x3'])
data_nm1['x4'] = np.log(data_nm1['x4'])
data_nm1['x5'] = np.log(data_nm1['x5'])

data_nm1['y1'] = np.log(data_nm1['y1'])
data_nm1['y2'] = np.log(data_nm1['y2'])

**This step can take a long time.**

In [ ]:
X = data_nm1[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm1[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

8.305100187417185

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 8.305100187417185
Beta values:
[[ 4.24407642e-01  9.19228230e-01 -1.83308583e-02  1.39424365e-01
  -4.27318804e-02]
 [ 1.79095648e-01  8.50325176e-02  1.39617787e-03  1.83790729e-02
   5.97255814e-02]
 [ 1.32647535e-01  2.30710221e-01  1.33360064e-03  6.37619860e-03
   7.65496021e-02]
 [ 2.69589815e-01  2.45463165e-01 -7.88288533e-04  3.26207990e-02
   3.19679261e-02]
 [ 3.88274684e-01  2.34984382e-01  9.42597893e-04  2.78823336e-03
   8.07657252e-02]]


In [ ]:
Y_hat = cp,matmul(X, beta) # Estimates of Y

In [ ]:
errors = compute_error(X, Y, beta)

Total sum of squares of error: [1.32631530e+01 6.87769259e+01 4.12540399e-02 5.46186173e+00
 9.56625064e+00]
Root mean square of error: [0.00313954 0.0071493  0.0001751  0.00201471 0.00266632]


In [ ]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 1', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 1a

We fit the model:

\begin{eqnarray}
\ln y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 + \beta_{10}\\
\ln y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 + \beta_{20} \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 + \beta_{30} \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 + \beta_{40} \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 + \beta_{50} \\
\end{eqnarray}

In [ ]:
data_nm1 = data.copy()

data_nm1['x2'] = np.log(data_nm1['x2'])
data_nm1['x3'] = np.log(data_nm1['x3'])
data_nm1['x4'] = np.log(data_nm1['x4'])
data_nm1['x5'] = np.log(data_nm1['x5'])

data_nm1['y1'] = np.log(data_nm1['y1'])
data_nm1['y2'] = np.log(data_nm1['y2'])

X = data_nm1[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm1[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X1, beta)))
prob = cp.Problem(obj)
prob.solve()

8.131718387558735

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 8.131718387558735
Beta values:
[[ 8.90788558e-02  5.74988290e-01 -9.50026868e-03  1.16636348e-01
  -1.42652441e-01]
 [ 1.13444977e-02 -8.72357366e-02  1.85781502e-03  7.74110256e-03
   8.80722709e-03]
 [-2.17434418e-02  7.21693077e-02 -5.44201275e-05 -3.12573944e-03
   2.94501552e-02]
 [ 2.82224982e-02 -2.56561620e-03 -2.14124802e-03  1.77753164e-02
  -4.20399104e-02]
 [ 4.09692015e-02 -1.22079097e-01  8.09385983e-04 -1.89398338e-02
  -2.47706318e-02]
 [ 5.23364643e+00  5.37800571e+00  1.26805378e-02  3.25917593e-01
   1.59519489e+00]]


In [ ]:
Y_hat = cp,matmul(X1, beta) # Estimates of Y

In [ ]:
compute_error(X1, Y, beta)

Total sum of squares of error: [1.07211918e+01 6.60971778e+01 4.13595123e-02 5.45155239e+00
 9.33025642e+00]
Root mean square of error: [0.00282269 0.00700864 0.00017532 0.00201281 0.00263323]


array([0.00282269, 0.00700864, 0.00017532, 0.00201281, 0.00263323])

In [ ]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 1a', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 2

Transform the variables. We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 \\
\end{eqnarray}

In [ ]:
data_nm2 = data.copy()

data_nm2['x2'] = np.log(data_nm2['x2'])
data_nm2['x3'] = np.log(data_nm2['x3'])
data_nm2['x4'] = np.log(data_nm2['x4'])
data_nm2['x5'] = np.log(data_nm2['x5'])

**This step can take a few minutes.**

In [ ]:
X = data_nm2[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm2[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

918.4829557994454

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 918.4829557994454
Beta values:
[[ 3.25715798e+01  5.62077151e+01 -3.60577321e-01 -5.80506727e-01
  -1.25713030e+00]
 [ 8.51565325e+00 -2.20178778e+00 -1.87710797e-01 -1.13926160e-01
  -5.18954634e-03]
 [-1.26475186e+00  1.18470501e+01  1.30673201e-01  4.18812154e-02
   2.99929555e-01]
 [ 1.55160765e+01  7.75954533e+00  1.75203901e-01  5.82633512e-02
  -1.17186472e-01]
 [ 2.31772235e+01  3.30955793e-01 -1.41628782e-01  7.27569643e-02
   1.71138948e-01]]


In [ ]:
Y_hat = cp,matmul(X, beta) # Estimates of Y

In [ ]:
errors = compute_error(X, Y, beta)

Total sum of squares of error: [8.39074151e+05 6.48633384e+05 1.35620404e+00 6.06658613e+00
 1.14750340e+01]
Root mean square of error: [0.78966371 0.6942912  0.00100393 0.00212331 0.00292024]


In [ ]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 2', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Linear model using Nelder-Mead algorithm

We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5
\end{eqnarray}

In [9]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

def error_fn0(beta):
  return np.linalg.norm(Y[:, 0] - X @ beta)

def error_fn1(beta):
  return np.linalg.norm(Y[:, 1] - X @ beta)

def error_fn2(beta):
  return np.linalg.norm(Y[:, 2] - X @ beta)

def error_fn3(beta):
  return np.linalg.norm(Y[:, 3] - X @ beta)

def error_fn4(beta):
  return np.linalg.norm(Y[:, 4] - X @ beta)

beta = np.zeros((5, 5))

def run_nelder_mead_solver(beta):
  r0 = optimize.minimize(error_fn0, beta[0, :], method='nelder-mead')
  r1 = optimize.minimize(error_fn1, beta[1, :], method='nelder-mead')
  r2 = optimize.minimize(error_fn2, beta[2, :], method='nelder-mead')
  r3 = optimize.minimize(error_fn3, beta[3, :], method='nelder-mead')
  r4 = optimize.minimize(error_fn4, beta[4, :], method='nelder-mead')

  beta[0, :] = r0.fun
  beta[1, :] = r1.fun
  beta[2, :] = r2.fun
  beta[3, :] = r3.fun
  beta[4, :] = r4.fun

  return beta

beta = run_nelder_mead_solver(beta)


In [10]:
Y_hat = X @ beta # Get estimates of Y.

In [ ]:
errors = compute_error_orig(X, Y, beta, scaler)

[6.07828666e+01 5.34628055e+01 1.35100176e-02 1.55170923e-01
 2.03375578e-01]


In [ ]:
model_summary = model_summary.append({'model_name': 'Nelder-Mead', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Levenberg-Marquardt algorithm

The <code>scipy</code> function <code>leastsq</code> used the Levenberg-Marquardt algorithm to find the values of the coefficients that minimize the squared error. Refer to the notes section of the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.leastsq.html#scipy.optimize.leastsq) and the MINPACK documentation of [lmder](https://www.math.utah.edu/software/minpack/minpack/lmder.html) and [lmdif](https://www.math.utah.edu/software/minpack/minpack/lmdif.html).


In [ ]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

def error_fn0(beta):
  return (Y[:, 0] - X @ beta)

def error_fn1(beta):
  return (Y[:, 1] - X @ beta)

def error_fn2(beta):
  return (Y[:, 2] - X @ beta)

def error_fn3(beta):
  return (Y[:, 3] - X @ beta)

def error_fn4(beta):
  return (Y[:, 4] - X @ beta)

beta = np.zeros((5, 5))

def run_lm_solver(beta):
  r0 = optimize.leastsq(error_fn0, beta[0, :])
  r1 = optimize.leastsq(error_fn1, beta[1, :])
  r2 = optimize.leastsq(error_fn2, beta[2, :])
  r3 = optimize.leastsq(error_fn3, beta[3, :])
  r4 = optimize.leastsq(error_fn4, beta[4, :])

  beta[0, :] = r0[0]
  beta[1, :] = r1[0]
  beta[2, :] = r2[0]
  beta[3, :] = r3[0]
  beta[4, :] = r4[0]

  return beta

beta = run_lm_solver(beta)

In [ ]:
Y_hat = X @ beta # Get estimates of Y.

In [ ]:
errors = compute_error_orig(X, Y, beta, scaler)

[7.90030729e-01 6.93673148e-01 1.75192641e-04 2.01462603e-03
 2.64331028e-03]


In [ ]:
model_summary = model_summary.append({'model_name': 'Levenberg-Marquardt', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

In [ ]:
model_summary

,model_name,err_y1,err_y2,err_y3,err_y4,err_y5
0,Linear model - 1,0.788313,0.692319,0.000175,0.002013,0.002633
1,Linear model - 2,0.788313,0.692320,0.000175,0.002013,0.002633
2,Nonlinear model - 1,0.003140,0.007149,0.000175,0.002015,0.002666
3,Nonlinear model - 1a,0.003140,0.007149,0.000175,0.002015,0.002666
4,Nonlinear model - 2,0.789664,0.694291,0.001004,0.002123,0.002920
5,Nelder-Mead,60.782867,53.462805,0.013510,0.155171,0.203376
6,Levenberg-Marquardt,0.790031,0.693673,0.000175,0.002015,0.002643


# Conclusion
The model summary indicates that the non-linear model has given the best results on simulated data. The exercise needs to repeated on real data to get the best results.

# Weighted model - 1

In [ ]:
weights = 0.2 * np.ones(5)
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]
y = np.average(Y, axis=1, weights=weights)
y = y.reshape((n_obs, 1))

In [ ]:
beta = cp.Variable((5, 1))
obj = cp.Minimize(cp.sum_squares(y - X@beta))
prob = cp.Problem(obj)
prob.solve()

222.43462549212643

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 222.43462549212643
Beta values:
[[-0.00100429]
 [ 0.00412584]
 [ 0.01277479]
 [-0.00303544]
 [-0.00735161]]


In [ ]:
Y_hat = cp.matmul(X, beta) # Estimates of Y.

In [ ]:
errors = compute_error(X, y, beta)

Total sum of squares of error: [222.43462549]
Root mean square of error: [0.01285711]


In [ ]:
model_summary = model_summary.append({'model_name': 'Weighted model - 1', 'err_y1': errors[0], 'err_y2': None, 'err_y3': None, 'err_y4': None, 'err_y5': None}, ignore_index=True)

# Weighted model - 1a

In [ ]:
weights = np.ones(5)/5
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]
y = np.average(Y, axis=1, weights=weights)
y = y.reshape((n_obs, 1))

In [ ]:
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 1))
obj = cp.Minimize(cp.sum_squares(y - X1@beta))
prob = cp.Problem(obj)
prob.solve()

222.4346254921264

In [ ]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 222.4346254921264
Beta values:
[[-1.00429183e-03]
 [ 4.12584027e-03]
 [ 1.27747863e-02]
 [-3.03544212e-03]
 [-7.35160715e-03]
 [ 5.09418662e-18]]


In [ ]:
Y_hat = cp.matmul(X, beta) # Estimates of Y.

In [ ]:
errors = compute_error(X1, y, beta)

Total sum of squares of error: [222.43462549]
Root mean square of error: [0.01285711]


In [ ]:
model_summary = model_summary.append({'model_name': 'Weighted model - 1a', 'err_y1': errors[0], 'err_y2': None, 'err_y3': None, 'err_y4': None, 'err_y5': None}, ignore_index=True)